In [1]:
import pandas  as pd
from langchain.llms import OpenAI
from dotenv import load_dotenv

import openai,os
load_dotenv(r'D:\Git\NLP\LLM\ActivLoop\.env')
openai_api_key = os.getenv("ACTIVELOOP_TOKEN")

assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
openai.api_key = openai_api_key

openai.api_base = os.getenv("OpenAiService")
openai.api_type = "azure"
openai.api_version =os.getenv("OpenAiVersion")
davincimodel= os.getenv("OpenAiDavinci")
active_loop_token=os.getenv("ACTIVELOOP_TOKEN")
embedding_model=os.getenv("OpenAiEmbedding")
chat_ai=os.getenv("ChatAI")#
HUGGINGFACEHUB_API_TOKEN=os.getenv("HUGGINGFACEHUB_API_TOKEN")

c:\Users\sri.karan\.conda\envs\activeloop\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.17) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


### Introduction
As we witness accelerated technological progress, large language models like GPT-4 and ChatGPT have emerged as significant breakthroughs in the tech landscape. These state-of-the-art models demonstrate exceptional prowess in content generation. However, they are not without their share of challenges, such as biases and hallucinations. Despite these limitations, LLMs have the potential to bring about a transformative impact on chatbot development.

Traditional, primarily intent-based chatbots have been designed to respond to specific user intents. These intents comprise a collection of sample questions and corresponding responses. For instance, a "Restaurant Recommendations" intent might include sample questions like "Can you suggest a good Italian restaurant nearby?" or "Where can I find the best sushi in town?" with responses such as "You can try the Italian restaurant 'La Trattoria' nearby" or "The top-rated sushi place in town is 'Sushi Palace.'"

When users interact with the chatbot, their queries are matched to the most similar intent, generating the associated response. However, as LLMs continue to evolve, chatbot development is shifting toward more sophisticated and dynamic solutions capable of handling a broader range of user inquiries with greater precision and nuance.

### Having a Knowledge Base
LLMs can significantly enhance chatbot functionality by associating broader intents with documents from a Knowledge Base (KB) instead of specific questions and answers. This approach streamlines intent management and generates more tailored responses to user inquiries.

GPT3 has a maximum prompt size of around 4,000 tokens, which is substantial but insufficient for incorporating an entire knowledge base in a single prompt. 

Future LLMs may not have this limitation while retaining the text generation capabilities. However, for now, we need to design a solution around it.

##### Workflow
This project aims to build a chatbot that leverages GPT3 to search for answers within documents. The workflow for the experiment is explained in the following diagram.

First we scrape some content from online articles, we split them into small chunks, compute their embeddings and store them in Deep Lake. Then, we use a user query to retrieve the most relevant chunks from Deep Lake, we put them into a prompt, which will be used to generate the final answer by the LLM.

It is important to note that there is always a risk of generating hallucinations or false information when using LLMs. Although this might not be acceptable for many customers support use cases, the chatbot can still be helpful for assisting operators in drafting answers that they can double-check before sending them to the user.

In the next steps, we'll explore how to manage conversations with GPT-3 and provide examples to demonstrate the effectiveness of this workflow:

First, set up the OPENAI_API_KEY and ACTIVELOOP_TOKEN environment variables with your API keys and tokens.

As we’re going to use the SeleniumURLLoader LangChain class, and it uses the unstructured and selenium Python library, let’s install it using pip. It is recommended to install the latest version of the library. Nonetheless, please be aware that the code has been tested specifically on version 0.7.7.

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI
from langchain.document_loaders import SeleniumURLLoader
from langchain import PromptTemplate

These libraries provide functionality for handling OpenAI embeddings, managing vector storage, splitting text, and interacting with the OpenAI API. They also enable the creation of a context-aware question-answering system, incorporating retrieval and text generation.

The database for our chatbot will consist of articles regarding technical issues.

In [3]:
# we'll use information from the following articles
urls = ['https://beebom.com/what-is-nft-explained/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-download-gif-twitter/',
        'https://beebom.com/how-use-chatgpt-linux-terminal/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-save-instagram-story-with-music/',
        'https://beebom.com/how-install-pip-windows/',
        'https://beebom.com/how-check-disk-usage-linux/']

### 1: Split the documents into chunks and compute their embeddings
We load the documents from the provided URLs and split them into chunks using the CharacterTextSplitter with a chunk size of 1000 and no overlap:

In [4]:
# use the selenium scraper to load the documents
loader = SeleniumURLLoader(urls=urls)
docs_not_splitted = loader.load()



In [6]:
print(docs_not_splitted[0].page_content)

Home  Internet  NFTs Explained: What is an NFT and What is Its Use

NFTs Explained: What is an NFT and What is Its Use

Arjun Sha

Last Updated: December 6, 2021 4:42 pm

After Bitcoin and Blockchain, NFT is another word to have entered our lexicon. The buzzword is everywhere and people are wondering what is NFT and what is its use? Well, there is not really a one-line explainer. And that’s why we have brought a comprehensive explainer on NFT, what is its use in digital art, and more. So without wasting any time, let’s go ahead and learn about NFTs (Non-fungible Token) in complete detail.

What is NFT: A Definitive Explainer (2021)

Here, we have mentioned all the questions that people have in their minds regarding NFT. You can click on the table to find all the sections that we have covered in this article and click on the link to move to the corresponding section.

Table of Contents

NFTs Explained: What is NFT in Crypto?

What is the Use of NFT in Digital Art?

How is Blockchain Rel

In [7]:
# we split the documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs_not_splitted)

Created a chunk of size 1226, which is longer than the specified 1000


In [8]:
len(docs)

127

Next, we compute the embeddings using OpenAIEmbeddings and store them in a Deep Lake vector store on the cloud. In an ideal production scenario, we could upload a whole website or course lesson on a Deep Lake dataset, allowing for search among even thousands or millions of documents. As we are using a cloud serverless Deep Lake dataset, applications running on different locations can easily access the same centralized dataset without the need of deploying a vector store on a custom machine.

Let’s now modify the following code by adding your Activeloop organization ID. It worth noting that the org id is your username by default.

# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "hayagriva99999"
my_activeloop_dataset_name = "langchain_course_customer_support"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

In [11]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
embeddings = OpenAIEmbeddings(deployment=embedding_model)

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "hayagriva99999"
my_activeloop_dataset_name = "langchain_course_customer_support"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset


Deep Lake Dataset in hub://hayagriva99999/langchain_course_customer_support already exists, loading from the storage


In [13]:
len(docs)

127

In [14]:
db.add_documents(docs[:16])

-

Dataset(path='hub://hayagriva99999/langchain_course_customer_support', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (16, 1536)  float32   None   
    id        text      (16, 1)      str     None   
 metadata     json      (16, 1)      str     None   
   text       text      (16, 1)      str     None   


['5ef2decd-34ec-11ee-8522-70cd0d7e92f2',
 '5ef2dece-34ec-11ee-b97b-70cd0d7e92f2',
 '5ef2decf-34ec-11ee-acc1-70cd0d7e92f2',
 '5ef2ded0-34ec-11ee-bb13-70cd0d7e92f2',
 '5ef2ded1-34ec-11ee-b514-70cd0d7e92f2',
 '5ef2ded2-34ec-11ee-a299-70cd0d7e92f2',
 '5ef2ded3-34ec-11ee-abbe-70cd0d7e92f2',
 '5ef2ded4-34ec-11ee-990a-70cd0d7e92f2',
 '5ef2ded5-34ec-11ee-b59f-70cd0d7e92f2',
 '5ef2ded6-34ec-11ee-ad23-70cd0d7e92f2',
 '5ef2ded7-34ec-11ee-a192-70cd0d7e92f2',
 '5ef2ded8-34ec-11ee-91a3-70cd0d7e92f2',
 '5ef2ded9-34ec-11ee-a165-70cd0d7e92f2',
 '5ef2deda-34ec-11ee-b3f1-70cd0d7e92f2',
 '5ef2dedb-34ec-11ee-8f12-70cd0d7e92f2',
 '5ef2dedc-34ec-11ee-b65d-70cd0d7e92f2']

To retrieve the most similar chunks to a given query, we can use the similarity_search method of the Deep Lake vector store:

In [15]:
# let's see the top relevant documents to a specific query
query = "how to check disk usage in linux?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Uniqueness is the key here. A Non-fungible item is, therefore, something that is unique and authentic. Something that holds value for its authenticity. Now we come to the token part. A token is a 40-digit string of words and numbers that hold the information of the original artist who created the masterpiece, the current owner of the item (it can only be one), and the current price of the item. This storage of information is backed by Blockchain technology which can’t be altered by any means due to its decentralized system.

Finally, when you put all three words together, NFT means an incorruptible token that holds ownership information about an item that is unique and authentic. NFT is basically a certificate of ownership or authenticity. This is the general definition of NFT, but if you want to get a complete idea about NFTs then you need to understand the concept with digital art as an example, which I have covered in the next section.

What is the Use of NFT in Digital Art?


### 2: Craft a prompt for GPT-3 using the suggested strategies
We will create a prompt template that incorporates role-prompting, relevant Knowledge Base information, and the user's question:

In [16]:
# let's write a prompt for a customer support chatbot that
# answer questions using information extracted from our db
template = """You are an exceptional customer support chatbot that gently answer questions.

You know the following context information.

{chunks_formatted}

Answer to the following question from a customer. Use only information from the previous context information. Do not invent stuff.

Question: {query}

Answer:"""

prompt = PromptTemplate(
    input_variables=["chunks_formatted", "query"],
    template=template,
)

The template sets the chatbot's persona as an exceptional customer support chatbot. The template takes two input variables: chunks_formatted, which consists of the pre-formatted chunks from articles, and query, representing the customer's question. The objective is to generate an accurate answer using only the provided chunks without creating any false or invented information.

### 3: Utilize the GPT3 model with a temperature of 0 for text generation
To generate a response, we first retrieve the top-k (e.g., top-3) chunks most similar to the user query, format the prompt, and send the formatted prompt to the GPT3 model with a temperature of 0.

In [18]:
# the full pipeline

# user question
query = "How to check disk usage in linux?"

# retrieve relevant chunks
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, query=query)

# generate answer
llm = OpenAI(engine=davincimodel, temperature=0)
answer = llm(prompt_formatted)
print(answer)


WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


 To check disk usage in Linux, you can use the command line utility 'df'. This command will show you the total disk space, used disk space, and available disk space. You can also use the 'du' command to check the disk usage of individual files and directories.


### Issues with Generating Answers using GPT-3
In the previous example, the chatbot generally performs well. However, there are certain situations where it could fail.

Suppose we ask, "Is the Linux distribution free?" and provide GPT-3 with a document about kernel features as context. It might generate an answer like "Yes, the Linux distribution is free to download and use," even if such information is not present in the context document. Producing false information is highly undesirable for customer service chatbots!

GPT-3 is less likely to generate false information when the answer to the user's question is contained within the context. Since user questions are often brief and ambiguous, we cannot always rely on the semantic search step to retrieve the correct document. Thus, there is always a risk of generating false information.

### Conclusion
GPT-3 is highly effective in creating conversational chatbots capable of answering specific questions based on the contextual information provided in the prompt. However, it can be challenging to ensure that the model generates answers solely based on the context, as it has a tendency to hallucinate (i.e., generate new, potentially false information). The severity of generating false information varies depending on the use case.

To conclude, we implemented a context-aware question-answering system using LangChain, following the provided code and strategies. The process involved splitting documents into chunks, computing their embeddings, implementing a retriever to find similar chunks, crafting a prompt for GPT-3, and using the GPT3 model for text generation. This approach demonstrates the potential of leveraging GPT-3 to create powerful and contextually accurate chatbots while also highlighting the need to be cautious about the possibility of generating false information.

In the next lesson, you’ll see how to leverage Deep Lake and GPT-4 for building a sales assistant.

https://learnprompting.org/docs/applied_prompting/build_chatbot_from_kb

In [2]:
import os

from dotenv import load_dotenv
import streamlit as st

# from api_utils import BuildBook
# from deep_lake_utils import SaveToDeepLake
# from pdf_gen_utils import build_pdf
import sys
sys.path.append(r"c:\users\sri.karan\appdata\roaming\python\python38\site-packages")

STYLES = {
            'Impressionism': 'Monet, impressionist art style, loose brushstrokes, vibrant colors, painted, painted light',
            'Cubism': 'Cubist art style, Picasso, fragmented forms, geometric shapes, angular lines, limited color palette, artistic',
            'Surrealism': 'Surrealist art style, dreamlike, abstract art, dream-like artwork, Salvador Dalí, art',
            'Japanese Ukiyo-e': 'Ukiyo-e art style, Hokusai, woodblock prints, flat areas of color, outlines, nature, Japanese culture',
            'Art Nouveau': 'Art Nouveau style, Mucha, curving lines, natural forms, ornamental, elegant, stylized',
            'Folk Art': 'Folk art style, naive art, simple shapes, bright colors, childlike, intuitive, traditional',
            'Expressionism': 'Expressionist art style, Edvard Munch, distorted forms, dramatic colors, emotional impact, subjective'
          }


load_dotenv(r'D:\Git\NLP\LLM\ActivLoop\.env')
dataset_path = os.getenv('DATASET_PATH')
